# GenoPhenoCorr Example 
## Phenotypic Traits associate with Variants on Gene MAPK8IP3


### TODO - Intro to package
BLAH BLAH BAahh

In [1]:
import sys
import glob
  
from genophenocorr import *
from genophenocorr import compare
from genophenocorr import compare_func as CF

## Importing Phenopacket Data

The phenopackets were created using [pyphetools](https://github.com/monarch-initiative/pyphetools) with data from [Platzer K., et al. (2019) De Novo Variants in MAPK8IP3 Cause Intellectual Disability with Variable Brain Anomalies](https://pubmed.ncbi.nlm.nih.gov/30612693/) and [Iwasawa S., et al. (2019) De Recurrent de novo MAPK8IP3 variants cause neurological phenotypes](https://pubmed.ncbi.nlm.nih.gov/30945334/). 


In [2]:
allPatients = Cohort('../phenopackets/PlatzerData/*.json')

INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/rekerl/Library/Caches/pyensembl/GRCh38/ensembl107/Homo_sapiens.GRCh38.pep.all.fa.gz.pickle


You can also add additional phenopackets to the AllPatients class by using AllPatients.add("Path/to/phenopacket.json"). 

In [3]:
##TO DO - use this to add the additional patient Ben(I think) found
# allPatients.add("json file to last subject")

## Examine the Data


In [4]:
allPatients.describe_all()

,Patient ID,Disease,Gene,Variant,Protein,HPO Terms
0,Individual 5,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0001270, HP:0032988, HP:0025336, HP:003193..."
1,12,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0001257, HP:0100021, HP:0010864]"
2,2,None,[MAPK8IP3],chr16 g.1706418G>T,ENSP00000501096,"[HP:0001251, HP:0010864]"
3,3,None,[MAPK8IP3],chr16 g.1706450C>G,ENSP00000501096,[HP:0002342]
4,13,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0002317, HP:0007301, HP:0002370, HP:0002342]"
5,Individual 4,None,[MAPK8IP3],chr16 g.1767834C>T,ENSP00000501096,"[HP:0001270, HP:0032988, HP:0032989, HP:002533..."
6,4,None,[MAPK8IP3],chr16 g.1748705G>A,ENSP00000501096,[HP:0001256]
7,8,None,[MAPK8IP3],chr16 g.1762843C>T,ENSP00000501096,"[HP:0001258, HP:0010864]"
8,Individual 3,None,[MAPK8IP3],chr16 g.1762843C>T,ENSP00000501096,"[HP:0001270, HP:0025336, HP:0031936, HP:000218..."
9,Individual 2,None,[MAPK8IP3],chr16 g.1762843C>T,ENSP00000501096,"[HP:0001270, HP:0032989, HP:0031936, HP:001086..."


We can also look more in depth as specific attributes of a patient. 

In [5]:
firstPat = allPatients.all_patients.get('1')
firstPat.describe()

ID                                                                   1
Disease ID                                                        None
Disease Label                                                     None
HPO IDs                                   ['HP:0001251', 'HP:0002342']
HPO Terms              ['Ataxia', 'Intellectual disability, moderate']
Variant                                    chr16 g.1706403_1706403delG
Variant Type                                     ['deletion', 'indel']
Gene Affected                                                 MAPK8IP3
Gene ID                                                ENSG00000138834
Effect of Variant                                              p.G22fs
Transcript ID                                          ENST00000673691
Protein Affected     C-Jun-amino-terminal kinase-interacting protein 3
Protein ID                                             ENSP00000501096
dtype: object

## Run Correlation Analysis
This function runs either a Chi^2^ test or a Fisher Exact test on the counts of each HPO Term that do or don't match a specific criteria. 
Format for function:
RunStats(AllPatients Class, Fun1, Fun2, extraVar_1, extraVar_2)

If Fun1 is CF.is_var_type, then extraVar_1 can be any variant type (missense, nonsense, deletion, etc.) and the function will count by HPO Term all the patients that have that type of variant and do or do not have that HPO Term. This will create the first half of the Contingency Table. Fun2 and extraVar_2 will do the same, creating the second half of the table. 

The example below creates a contingency table: <br>

| Patients with HPO Term and with missense variant | Patients without HPO Term and with missense variant | <br>
| Patients with HPO Term and with no missense variant | Patients without HPO Term and with no missense variant |


In [6]:
compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, 'missense', 'missense')


Total hpo terms: 36
BonCorrected p-value = 0.001388888888888889
                                              1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0001251-Ataxia                                  0.0       15.0       2.0   
HP:0002342-Intellectual disability, moderate       4.0       11.0       2.0   
HP:0001270-Motor delay                             5.0       10.0       0.0   
HP:0031936-Delayed ability to walk                 5.0       10.0       0.0   
HP:0000219-Thin upper lip vermilion                5.0       10.0       0.0   

                                              2 w/o hpo      pval  
HP:0001251-Ataxia                                   1.0  0.019608  
HP:0002342-Intellectual disability, moderate        1.0  0.245098  
HP:0001270-Motor delay                              3.0  0.522059  
HP:0031936-Delayed ability to walk                  3.0  0.522059  
HP:0000219-Thin upper lip vermilion                 3.0  0.522059  


,1 w/ hpo,1 w/o hpo,2 w/ hpo,2 w/o hpo,pval
HP:0001251-Ataxia,0.0,15.0,2.0,1.0,0.019608
"HP:0002342-Intellectual disability, moderate",4.0,11.0,2.0,1.0,0.245098
HP:0001270-Motor delay,5.0,10.0,0.0,3.0,0.522059
HP:0031936-Delayed ability to walk,5.0,10.0,0.0,3.0,0.522059
HP:0000219-Thin upper lip vermilion,5.0,10.0,0.0,3.0,0.522059
HP:0002317-Unsteady gait,1.0,14.0,0.0,3.0,1.000000
HP:0007301-Oromotor apraxia,1.0,14.0,0.0,3.0,1.000000
HP:0002370-Poor coordination,1.0,14.0,0.0,3.0,1.000000
HP:0032989-Delayed ability to roll over,2.0,13.0,0.0,3.0,1.000000
HP:0004322-Precocious puberty,4.0,11.0,0.0,3.0,1.000000


In [7]:
for typ in allPatients.all_var_types:
    print(f'Comparing all in {typ} vs out of {typ}')
    compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, typ, typ)
    for typ2 in allPatients.all_var_types:
        if typ2 == typ:
            continue
        print(f'Comparing all in {typ} vs all in {typ2}')
        compare.RunStats(allPatients, CF.is_var_type, CF.is_var_type, typ, typ2)

Comparing all in transversion vs out of transversion
Total hpo terms: 36
BonCorrected p-value = 0.001388888888888889
                                              1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0002342-Intellectual disability, moderate       2.0        1.0       4.0   
HP:0001251-Ataxia                                  1.0        2.0       1.0   
HP:0001270-Motor delay                             0.0        3.0       5.0   
HP:0031936-Delayed ability to walk                 0.0        3.0       5.0   
HP:0000219-Thin upper lip vermilion                0.0        3.0       5.0   

                                              2 w/o hpo      pval  
HP:0002342-Intellectual disability, moderate       11.0  0.245098  
HP:0001251-Ataxia                                  14.0  0.313725  
HP:0001270-Motor delay                             10.0  0.522059  
HP:0031936-Delayed ability to walk                 10.0  0.522059  
HP:0000219-Thin upper lip vermilion                10.0  0.522059  


                                              1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0001270-Motor delay                             5.0       10.0       5.0   
HP:0001251-Ataxia                                  0.0       15.0       0.0   
HP:0002342-Intellectual disability, moderate       4.0       11.0       3.0   
HP:0002317-Unsteady gait                           1.0       14.0       1.0   
HP:0007301-Oromotor apraxia                        1.0       14.0       1.0   

                                              2 w/o hpo  pval  
HP:0001270-Motor delay                              9.0   1.0  
HP:0001251-Ataxia                                  14.0   1.0  
HP:0002342-Intellectual disability, moderate       11.0   1.0  
HP:0002317-Unsteady gait                           13.0   1.0  
HP:0007301-Oromotor apraxia                        13.0   1.0  
Comparing all in missense vs all in nonsense
Total hpo terms: 36
BonCorrected p-value = 0.001388888888888889
                                

                                              1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0001251-Ataxia                                  0.0       14.0       2.0   
HP:0002342-Intellectual disability, moderate       3.0       11.0       3.0   
HP:0000219-Thin upper lip vermilion                5.0        9.0       0.0   
HP:0001270-Motor delay                             5.0        9.0       0.0   
HP:0031936-Delayed ability to walk                 5.0        9.0       0.0   

                                              2 w/o hpo      pval  
HP:0001251-Ataxia                                   2.0  0.039216  
HP:0002342-Intellectual disability, moderate        1.0  0.083333  
HP:0000219-Thin upper lip vermilion                 4.0  0.277778  
HP:0001270-Motor delay                              4.0  0.277778  
HP:0031936-Delayed ability to walk                  4.0  0.277778  
Comparing all in transition vs all in transversion
Total hpo terms: 36
BonCorrected p-value = 0.001388888888888889
  

                                              1 w/ hpo  1 w/o hpo  2 w/ hpo  \
HP:0001270-Motor delay                             0.0        2.0       0.0   
HP:0001251-Ataxia                                  1.0        1.0       1.0   
HP:0002342-Intellectual disability, moderate       1.0        1.0       1.0   
HP:0002317-Unsteady gait                           0.0        2.0       0.0   
HP:0007301-Oromotor apraxia                        0.0        2.0       0.0   

                                              2 w/o hpo  pval  
HP:0001270-Motor delay                              1.0   1.0  
HP:0001251-Ataxia                                   0.0   1.0  
HP:0002342-Intellectual disability, moderate        0.0   1.0  
HP:0002317-Unsteady gait                            1.0   1.0  
HP:0007301-Oromotor apraxia                         1.0   1.0  
Comparing all in nonsense vs all in transition
Total hpo terms: 36
BonCorrected p-value = 0.001388888888888889
                              

In [8]:
for var in allPatients.all_variants:
    print(f'Comparing affecting {var.variant_string} vs all not')
    compare.RunStats(allPatients, CF.is_var_match, CF.is_not_var_match, var, var)

Comparing affecting chr16 g.1767834C>T vs all not
Total hpo terms: 36
BonCorrected p-value = 0.001388888888888889


ValueError: 'variant' argument must be string or varcode Variant class but was <class 'genophenocorr.variant.Variant'>

In [ ]:
compare.RunStats(allPatients,CF.in_feature, CF.not_in_feature,'Domain','Domain')

In [ ]:
Protein.findProtein('ENSP00000501096')

In [ ]:
compare.RunStats(allPatients, CF.is_var_type, CF.is_not_var_type, 'missense', 'missense')
